In [1]:
# Import libraries
import os
import glob
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import pandas as pd
import datetime as dt

In [2]:
# Set input directory, and change working directory
BAinDir = 'C:/Users/SASUKE/Desktop/Spring2021/COMP491/NDVI_BA_ALL/'  # Path to the Burn Area Files
NDVI_inDir = 'C:/Users/SASUKE/Desktop/Spring2021/COMP491/NDVI_BA_ALL/'  #Path to NDVI files
os.chdir(NDVI_inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(BAinDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [3]:
EVIFiles = glob.glob('MOD13A1.006__500m_16_days_NDVI_**.tif')             # Search for and create a list of EVI files
EVIqualityFiles =glob.glob('MOD13A1.006__500m_16_days_VI_Quality**.tif')  # Search the directory for the associated quality .tifs
EVIlut = glob.glob('MOD13A1-006-500m-16-days-VI-Quality-lookup.csv')                                        # Search for look up table 
EVI_v6_QA_lut = pd.read_csv(EVIlut[0])                                    # Read in the lut

# Include good quality based on MODLAND
EVI_v6_QA_lut = EVI_v6_QA_lut[EVI_v6_QA_lut['MODLAND'].isin(['VI produced with good quality', 'VI produced, but check other QA'])]

# Exclude lower quality VI usefulness
VIU =["Lowest quality","Quality so low that it is not useful","L1B data faulty","Not useful for any other reason/not processed"]
EVI_v6_QA_lut = EVI_v6_QA_lut[~EVI_v6_QA_lut['VI Usefulness'].isin(VIU)]

EVI_v6_QA_lut = EVI_v6_QA_lut[EVI_v6_QA_lut['Aerosol Quantity'].isin(['Low','Average'])]   # Include low or average aerosol
EVI_v6_QA_lut = EVI_v6_QA_lut[EVI_v6_QA_lut['Adjacent cloud detected'] == 'No' ]           # Include where adjacent cloud not detected
EVI_v6_QA_lut = EVI_v6_QA_lut[EVI_v6_QA_lut['Mixed Clouds'] == 'No' ]                      # Include where mixed clouds not detected
EVI_v6_QA_lut = EVI_v6_QA_lut[EVI_v6_QA_lut['Possible shadow'] == 'No' ]                   # Include where possible shadow not detected

EVIgoodQuality = list(EVI_v6_QA_lut['Value']) # Retrieve list of possible QA values from the quality dataframe

In [4]:
BAFiles = glob.glob('MCD64A1.006_Burn_Date_**.tif') # Search for and create a list of BA files
BAqualityFiles =glob.glob('MCD64A1.006_QA_**.tif')    # Search the directory for the associated quality .tifs
lut = glob.glob('MCD64A1-006-QA-lookup.csv')                     # Search for look up table 
v6_BAQA_lut = pd.read_csv(lut[0])     # Read in the lut

# Include good quality based on MODLAND
v6_BAQA_lut = v6_BAQA_lut[v6_BAQA_lut['Valid data'].isin([True])]

# Special circumstances unburned
SP =["Too few training observations or insufficient spectral separability between burned and unburned classes"]
v6_BAQA_lut = v6_BAQA_lut[~v6_BAQA_lut['Special circumstances unburned'].isin(SP)]
v6_BAQA_lut

BAgoodQuality = list(v6_BAQA_lut['Value']) # Retrieve list of possible QA values from the quality dataframe
BAVal = tuple(range(1, 367, 1))

In [5]:
def getBAFile(BAF, month, year):
    for x in BAF:
        # File name metadata:
        BAproductId = x.split('_')[0]                                     # First: product name
        BAlayerId = x.split(BAproductId + '_')[1].split('_doy')[0]        # Second: layer name
        BAyeardoy = x.split(BAlayerId+'_doy')[1].split('_aid')[0]         # Third: date
        file_year = dt.datetime.strptime(BAyeardoy, '%Y%j').year
        file_month = dt.datetime.strptime(BAyeardoy, '%Y%j').month
        if file_year==year  and file_month == month:
            return(x) 

In [6]:
NDVI_result = []
for i in range(0, len(EVIFiles)):
    EVI = gdal.Open(EVIFiles[i])                    # Read file in, starting with MOD13Q1 version 6
    EVIBand = EVI.GetRasterBand(1)                  # Read the band (layer)
    EVIData = EVIBand.ReadAsArray().astype('float') # Import band as an array with type float
    
    EVIquality = gdal.Open(EVIqualityFiles[i])                       # Open the first quality file
    EVIqualityData = EVIquality.GetRasterBand(1).ReadAsArray()       # Read in as an array
    EVIquality = None 
        
    # File name metadata:
    EVIproductId = EVIFiles[i].split('_')[0]                                      # First: product name
    EVIlayerId = EVIFiles[i].split(EVIproductId + '_')[1].split('_doy')[0]        # Second: layer name
    EVIyeardoy = EVIFiles[i].split(EVIlayerId+'_doy')[1].split('_aid')[0]         # Third: date
    EVIaid = EVIFiles[i].split(EVIyeardoy+'_')[1].split('.tif')[0]                # Fourth: unique ROI identifier (aid)
    EVIdate = dt.datetime.strptime(EVIyeardoy, '%Y%j').strftime('%m/%d/%Y')       # Convert YYYYDDD to MM/DD/YYYY
    EVI_year = dt.datetime.strptime(EVIyeardoy, '%Y%j').year
    EVI_month = dt.datetime.strptime(EVIyeardoy, '%Y%j').month   
    
    BAFileName = getBAFile(BAFiles, EVI_month, EVI_year)
    BAQualityFileName = getBAFile(BAqualityFiles, EVI_month, EVI_year)
    if not BAFileName in (None, ''):
        BA = gdal.Open(BAFileName)      # Read file in, starting with MCD64A1 version 6
        BABand = BA.GetRasterBand(1)                                 # Read the band (layer)
        BAData = BABand.ReadAsArray().astype('float')                # Import band as an array with type float  
        
        BAquality = gdal.Open(BAQualityFileName)   # Open the first quality file
        BAqualityData = BAquality.GetRasterBand(1).ReadAsArray()                # Read in as an array
        BAquality = None 

        # File Metadata
        BA_meta = BA.GetMetadata()                        # Store metadata in dictionary
        EVI_meta = EVI.GetMetadata()                      # Store metadata in dictionary

        # Band metadata
        BAFill = BABand.GetNoDataValue()                  # Returns fill value
        BAStats = BABand.GetStatistics(True, True)        # returns min, max, mean, and standard deviation
        BA = None                                         # Close the GeoTIFF file

        # Band metadata
        EVIFill = EVIBand.GetNoDataValue()                # Returns fill value
        EVIStats = EVIBand.GetStatistics(True, True)      # returns min, max, mean, and standard deviation
        EVI = None                                        # Close the GeoTIFF file

        BAscaleFactor = float(BA_meta['scale_factor'])    # Search the metadata dictionary for the scale factor 
        BAData[BAData == BAFill] = np.nan                 # Set the fill value equal to NaN for the array
        BAScaled = BAData * BAscaleFactor                 # Apply the scale factor using simple multiplication
        print(BAscaleFactor)  
        print(BAData)  
        EVIscaleFactor = float(EVI_meta['scale_factor'])  # Search the metadata dictionary for the scale factor 
        EVIData[EVIData == EVIFill] = np.nan              # Set the fill value equal to NaN for the array
        EVIScaled = EVIData * EVIscaleFactor              # Apply the scale factor using simple multiplication

        BA_masked = np.ma.MaskedArray(BAScaled, np.in1d(BAqualityData, BAgoodQuality, invert = True))    # Apply QA mask to the BA data
        EVI_masked = np.ma.MaskedArray(EVIScaled, np.in1d(EVIqualityData, EVIgoodQuality, invert = True))# Apply QA mask to the EVI data
        EVI_BA = np.ma.MaskedArray(EVI_masked, np.in1d(BA_masked, BAVal, invert = True)) # Mask array, include only BurnArea
        EVI_mean = np.mean(EVI_BA.compressed())
       
        NDVI_result.append([EVIdate,EVI_mean])
result_df = pd.DataFrame(NDVI_result, columns=["Date","NDVI"])

1.0
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
1.0
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
1.0
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
1.0
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
1.0
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
1.0
[[nan nan nan ..

In [7]:
result_df['Date'] = pd.to_datetime(result_df['Date'], format='%m/%d/%Y')
result_df = result_df.set_index('Date')
result_series = result_df.resample('D').mean()

In [8]:
result_series["NDVI"] = result_series["NDVI"].interpolate(method='linear')

In [9]:
result_series.to_csv('NDVI2018_2020.csv', index = True)   # Export statistics to CSV

In [10]:
#import seaborn as sns
# Use seaborn style defaults and set the default figure size
#sns.set(rc={'figure.figsize':(11, 4)})

In [11]:
#plt.figure(figsize=(16, 8))
#plt.plot(result_series['NDVI'], color='r')
##plt.xlabel('Dates',size=18)
#plt.ylabel('NDVI Values', size=18)
#plt.title('NDVI 2019-2020', fontsize=24)
#plt.savefig(r'C:\Users\SASUKE\Desktop\NDVI', transparent=True)
#plt.show()


In [12]:
#result_series['NDVI'].plot(color='r');